In [2]:
import zipfile 
import pandas as pd

zip_file_path = './sim_data/gt_1a_100.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Assume there is only one CSV file in the zip (you might need to modify this if there are multiple CSV files)
    csv_file_name = zip_ref.namelist()[0]

    # Read the CSV file directly from the zip file into a pandas DataFrame
    with zip_ref.open(csv_file_name) as csv_file:
        df = pd.read_csv(csv_file)
        
df

,time,voltage,stim
0,0.0000,-80.000000,0.0
1,0.0125,-80.001144,0.0
2,0.0250,-80.002170,0.0
3,0.0375,-80.003110,0.0
4,0.0500,-80.004005,0.0
...,...,...,...
7996,99.9500,-73.548450,0.0
7997,99.9625,-73.551926,0.0
7998,99.9750,-73.555400,0.0
7999,99.9875,-73.558870,0.0


In [5]:
 import numpy as np
from scipy.integrate import odeint
from scipy.optimize import minimize

def hodgkin_huxley_inverse(target_potential, time, known_impulse, parameters_guess):
    """
    Recover parameters of the Hodgkin-Huxley model using the adjoint method.

    Parameters:
    - target_potential: 1D array, the observed membrane potential over time.
    - time: 1D array, the time points corresponding to the observed potential.
    - known_impulse: 1D array, the known impulse applied during the experiment.
    - parameters_guess: List, initial guess for the model parameters [C_m, g_Na, g_K, g_L, E_Na, E_K, E_L].

    Returns:
    - optimized_parameters: List, the optimized parameters recovered by the adjoint method.
    """

    def model(y, t, I_stim, C_m, g_Na, g_K, g_L, E_Na, E_K, E_L):
        V, m, h, n = y

        # Hodgkin-Huxley equations
        dVdt = (I_stim(t) - g_Na * m**3 * h * (V - E_Na) - g_K * n**4 * (V - E_K) - g_L * (V - E_L)) / C_m
        dmdt = alpha_m(V) * (1 - m) - beta_m(V) * m
        dhdt = alpha_h(V) * (1 - h) - beta_h(V) * h
        dndt = alpha_n(V) * (1 - n) - beta_n(V) * n

        return [dVdt, dmdt, dhdt, dndt]

    def alpha_m(V):
        return 0.1 * (V + 40) / (1 - np.exp(-(V + 40) / 10))

    def beta_m(V):
        return 4.0 * np.exp(-(V + 65) / 18)

    def alpha_h(V):
        return 0.07 * np.exp(-(V + 65) / 20)

    def beta_h(V):
        return 1.0 / (1 + np.exp(-(V + 35) / 10))

    def alpha_n(V):
        return 0.01 * (V + 55) / (1 - np.exp(-(V + 55) / 10))

    def beta_n(V):
        return 0.125 * np.exp(-(V + 65) / 80)

    def objective_function(parameters, target_potential, time, known_impulse):
        C_m, g_Na, g_K, g_L, E_Na, E_K, E_L = parameters

        # Define the impulse function using the known impulse
        def I_stim(t):
            return np.interp(t, time, known_impulse, left=0, right=0)

        # Solve the Hodgkin-Huxley equations
        solution = odeint(model, [-65, 0.05, 0.6, 0.32], time, args=(I_stim, C_m, g_Na, g_K, g_L, E_Na, E_K, E_L))

        # Calculate the mean squared difference from the target potential
        mse = np.mean((solution[:, 0] - target_potential)**2)

        return mse

    def adjoint_model(parameters, target_potential, time, known_impulse):
        C_m, g_Na, g_K, g_L, E_Na, E_K, E_L = parameters

        # Define the impulse function using the known impulse
        def I_stim(t):
            return np.interp(t, time, known_impulse, left=0, right=0)

        # Compute the forward solution
        solution = odeint(model, [-65, 0.05, 0.6, 0.32], time, args=(I_stim, C_m, g_Na, g_K, g_L, E_Na, E_K, E_L))

        # Compute the difference between the simulated and target potentials
        error = solution[:, 0] - target_potential

        # Compute the adjoint variable (gradient of the objective function)
        adjoint_variable = 2 * error

        return adjoint_variable

    # Set bounds for the parameters (you might need to adjust these based on your problem)
    bounds = [(1e-6, 1), (1e-6, 100), (1e-6, 100), (1e-6, 10), (-100, 100), (-100, 100), (-100, 100)]

    # Perform the optimization using the adjoint method with bounds
    result = minimize(objective_function, parameters_guess,
                      args=(target_potential, time, known_impulse),
                      method='L-BFGS-B', jac=adjoint_model, bounds=bounds, options={'disp': True})

    # Extract the optimized parameters
    optimized_parameters = result.x

    return optimized_parameters

# Example usage:
# Set the target membrane potential, time, and known impulse
target_potential =(df['voltage'].to_numpy())  # Replace with your target potential
time = np.arange(0, 100, 0.0125)
known_impulse = (df['stim'].to_numpy())  # Replace with your known impulse

# Initial guess for the parameters [C_m, g_Na, g_K, g_L, E_Na, E_K, E_L]
initial_guess_parameters = [1, 50, 20, 0.5, 50, -77, -54.387]

# Recover parameters using the adjoint method
optimized_parameters = hodgkin_huxley_inverse(target_potential, time, known_impulse, initial_guess_parameters)

print("Optimized Parameters:", optimized_parameters)


ValueError: fp and xp are not of the same length.